# Import packages

In [122]:
import geopandas as gpd
import matplotlib.patches as mpatches
import contextily as ctx
import matplotlib.pyplot as plt
import pandas as pd
import os
#import functions as fn
import rasterio
from rasterio.plot import show
from shapely.ops import unary_union
from shapely.validation import make_valid
from shapely.ops import nearest_points
import numpy as np

### Set parameters

In [123]:
year = 2005 #2005 is the base year for the matching process 
crs =  'EPSG:3148'

## Load relevant files

# Fuzzy Matching

In [124]:
# import needed packages
from rapidfuzz.process import extractOne
from thefuzz import fuzz, process

In [ ]:
root_path = 'DISES/Proximity/data' #https://drive.google.com/open?id=18ttY1vG89yW1TT4UNwBPimaUsl0nKvqt&usp=drive_fs

# define path
os.chdir(root_path)

### Import required datasets

In [126]:
#Updated community forests
cfs_upd = gpd.read_file('Camboda CF updated data - Feb-2025/CF_KH_Updated_2.5.2025_EPSG32648.shp')

#Villages
vlgs = gpd.read_file('KHM-Villages/Villages.shp').to_crs(cfs_upd.crs)

In [ ]:
#Load NIS data set
#Parameters
nis_year = 2006

#Load NIS information

nis_raw_data_path = os.path.join(f'STATA DHS/Stata NIS data/NIS_Village/NIS{nis_year}_Village_Year_Coords.xlsx')
nis_mpi_data_path = os.path.join(f'STATA DHS/Stata NIS data/MPI_NIS/MPI_NIS_{nis_year}.xlsx')

nis_path = os.path.join(root_path, nis_mpi_data_path)
nis = pd.read_excel(nis_path)#, usecols=nis_columns)

In [128]:
#Replace village name with nis name
vlgs = vlgs.drop('VILLAGE', axis=1)

In [129]:
#Rename the village name variable to make the code run
nis = nis.rename({'Village':'VILLAGE'}, axis=1)

In [130]:
vlgs = vlgs.merge(nis[['VillGis', 'VILLAGE']], how='left', left_on='PHUMCODE', right_on='VillGis')

In [131]:
vlgs['VILLAGE'].isnull().sum()

883

In [132]:
13970-883

13087

### CFs dataset preprocessing

In [133]:
#Create the CF code for further identification
cfs_upd.rename({'ObjectID':'CF_CODE'}, axis=1, inplace=True)
# Keep only relevant columns
cfs_upd = cfs_upd[['CF_CODE','CF_Name_En', '#Villages','Village', 'Commune', 'geometry']].copy()
# Standarize the commune variable
cfs_upd['Commune'] = cfs_upd['Commune'].str.lower()
# Sort values by ObjectID
cfs_upd.sort_values('CF_CODE', inplace=True)
#Drop the previous index
cfs_upd = cfs_upd.reset_index(drop=True)
#Create a string 3 digit code to uniquely identify the CFs
cfs_upd['CF_CODE'] = cfs_upd['CF_CODE'].astype(str).str.zfill(3)

In [134]:
#Count community forests
cfs_upd_original = cfs_upd.copy()
print(f'There are {str(cfs_upd_original.nunique()['CF_CODE'])} community forests in the updated dataset.' )

There are 598 community forests in the updated dataset.


In [135]:
# Count only those with a village
cfs_with_village = cfs_upd[cfs_upd['Village']!='N/A'].nunique()['CF_CODE']
print(f'However, only {cfs_with_village} had a village assigned.' )

However, only 376 had a village assigned.


## Split villages in multiple columns

In [136]:
vlg_split = cfs_upd['Village'].str.split(',', expand=True) 
vlg_split.columns = [f'Village{i+1}' for i in range(vlg_split.shape[1])]
cfs_upd = cfs_upd.drop(columns=['Village'])

In [137]:
# Concatenate with original DataFrame, dropping the old 'Values' column
df_melted = pd.concat([cfs_upd, vlg_split], axis=1)

In [138]:
# Melt the DataFrame
df_melted = df_melted.melt(id_vars=['CF_CODE'], value_vars=['Village1', 'Village2', 'Village3', 'Village4', 'Village5', 'Village6',
       'Village7'], var_name='vil_type', value_name='vil_melt')

# Drop null values
df_melted = df_melted[df_melted['vil_melt'].notnull()].reset_index(drop=True).copy()

# lowercase village names
df_melted['vil_melt'] = df_melted['vil_melt'].str.lower()

In [139]:
villages_cfs_updated_datasets = df_melted[df_melted['vil_melt'] != 'n/a'].shape[0]
print(f'There are {villages_cfs_updated_datasets} unique villages in the CFs dataset' )

There are 428 unique villages in the CFs dataset


In [140]:
#Drop observations with no Village name
df_melted = df_melted[df_melted['vil_melt'] != 'n/a']

In [141]:
#Get relevant variables from the original data set
cfs_villages = cfs_upd.merge(df_melted, on='CF_CODE', how='left')

## Match only to villages within 5Km of a CF

The georreferenced village dataset did not have a column containing the Commune. For that reason, a sjoin with a dataset of communes downloaded from https://data.humdata.org was performed. 

In [142]:
#Create the 5 km buffer around the CFs
cfs_5k = cfs_upd.copy()
cfs_5k['geometry'] = cfs_5k['geometry'].buffer(5000)

#Save the file
cfs_5k.to_file('cfs_5k_buffer.shp')
cfs_5k = cfs_5k.to_crs(vlgs.crs)

In [143]:
#Sjoin the 5km buffer with the Villages data set to get only proximity villages
villages_5k = gpd.sjoin(
    vlgs[['PHUMCODE', 'VILLAGE', 'geometry']], 
    cfs_5k, 
    how="inner", 
    predicate="within"
)

In [144]:
f"{villages_5k.shape[0]} villages fall within the 5km of CFs"

'5809 villages fall within the 5km of CFs'

In [145]:
#Save the shapefile of villages within 5km
villages_5k.to_file('villages_5k.shp') 

/var/folders/wn/vzyjwy2x00q4stf523pr30jw0000gn/T/ipykernel_12908/2468266793.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  villages_5k.to_file('villages_5k.shp')
/Users/Daniel/miniconda3/envs/pymc_env/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'index_right' to 'index_righ'
  ogr_write(


In [146]:
#Simplify the name for the workflow
df = villages_5k.copy()

In [147]:
#keep only the necessary columns
df = df[['PHUMCODE', 'VILLAGE', 'geometry', 'CF_CODE']]

### Fuzzy matching

In [148]:
#The fuzzy matching is done with this 2 CFs. 
#df #All villages within 5 km of CFs
#cfs_villages #All villages in CFs updates dataset with their corresponding CFs

In [149]:
#Create the 2 dfs for matching
df1 = df[['CF_CODE', 'PHUMCODE', 'VILLAGE']]

df2 = cfs_villages[['CF_CODE', 'vil_melt']]

df2 = df2[~df2['vil_melt'].isnull()]

In [150]:
df1['VILLAGE'] = df1['VILLAGE'].str.lower()

/var/folders/wn/vzyjwy2x00q4stf523pr30jw0000gn/T/ipykernel_12908/1060850753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['VILLAGE'] = df1['VILLAGE'].str.lower()


In [151]:
#Merge the 2 dataframes with CF villages on the left on the left
df3 = df2.merge(df1, on='CF_CODE', how='left')

In [152]:
df3["similarity_score"] = df3.apply(lambda row: fuzz.ratio(row["VILLAGE"], row["vil_melt"]), axis=1)

In [153]:
df_best_matches = df3.loc[
    df3.groupby(["CF_CODE", "vil_melt"])["similarity_score"].idxmax()
].copy()

In [154]:
df_best_matches[df_best_matches['similarity_score']<80].sample(10).sort_values('similarity_score', ascending=False)

CF_CODE         vil_melt    PHUMCODE        VILLAGE  similarity_score
1284     184           roluos  10040602.0    srae roluos                71
1316     196      kampong pou  13020801.0  kampong chrey                67
1749     218      ou ba krang  15020111.0   kaoh krasang                61
3772     616  trapaeng traeng  10060201.0     changkrang                56
3255     462        svay chum  10030106.0    stueng thum                48
2270     272        phum buon  16040503.0       ou plong                47
2875     372        ou sandan  15031108.0     choar mean                42
3397     501            thmei   5060104.0        chambak                33
3251     462       russey keo  10030106.0    stueng thum                27
675      094             kyov         NaN            NaN                 0

In [156]:
import matplotlib.pyplot as plt

# Plot histogram
plt.figure(figsize=(10, 6))
df_best_matches['similarity_score'].hist(bins=100, edgecolor='black')

# Add vertical line at 70
plt.axvline(x=60, color='red', linestyle='--', linewidth=1.5, label='Threshold = 60')

# Improve appearance
plt.title('Distribution of Similarity Scores', fontsize=16, weight='bold')
plt.xlabel('Similarity Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.show()

In [157]:
#Only similarity scores higher that 60
df3 = df_best_matches[df_best_matches['similarity_score'] > 60]

In [158]:
df3['PHUMCODE'].nunique()

324

In [159]:
df3['CF_CODE'].nunique()

298

In [72]:
df3 = df3[['CF_CODE', 'PHUMCODE', 'VILLAGE']]

In [73]:
df3 = df3.merge(vlgs[['PHUMCODE', 'geometry']], on='PHUMCODE', how='left')

In [74]:
gdf = gpd.GeoDataFrame(df3)

In [75]:
#Save the matching villages gdf as a shapefile
#gdf.to_file('matched_villages.shp')

## Finding the "nearest" villages for non-matched CFs

In [76]:
#Get all CFs that have no matching village so far
cfs1 = cfs_upd[~cfs_upd['CF_CODE'].isin(df3['CF_CODE'])]

In [77]:
def find_closest_phumcode(gdf1, gdf2):
    """
    Find the PHUMCODE of the closest point from gdf2 to each polygon in gdf1.

    Parameters:
    gdf1 (GeoDataFrame): A GeoDataFrame containing polygon geometries.
    gdf2 (GeoDataFrame): A GeoDataFrame containing point geometries with a 'PHUMCODE' column.

    Returns:
    GeoDataFrame: The original gdf1 with an additional column 'closest_phumcode' 
                  containing the PHUMCODE of the nearest point from gdf2.
    
    Example:
    --------
    # Load example data
    gdf1 = gpd.read_file('polygons.shp')
    gdf2 = gpd.read_file('points.shp')

    # Find closest PHUMCODE
    result = find_closest_phumcode(gdf1, gdf2)
    """
    closest_phumcodes = []
    for _, polygon in gdf1.iterrows():
        # Find the index of the nearest point
        nearest_geom = gdf2.geometry.distance(polygon.geometry).idxmin()
        # Get the corresponding PHUMCODE
        closest_phumcode = gdf2.loc[nearest_geom, 'PHUMCODE']
        closest_phumcodes.append(closest_phumcode)
    
    # Save the closest PHUMCODE to gdf1
    gdf1['PHUMCODE'] = closest_phumcodes
    return gdf1


In [78]:
# Find closest PHUMCODE
result = find_closest_phumcode(cfs1, vlgs)

/Users/Daniel/miniconda3/envs/pymc_env/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [79]:
nearest_villages = result[['CF_CODE', 'PHUMCODE']].merge(vlgs, on='PHUMCODE', how='left')

In [80]:
#Unify format
nearest_villages['VILLAGE'] = nearest_villages['VILLAGE'].str.lower()

In [81]:
#gpd.GeoDataFrame(nearest_villages).to_file('closest_villages.shp')

# A unified treatment villages dataset

In [82]:
gdf['Criterion'] = 'Matching'

In [83]:
#Simpllified nearest
nearest_villages['Criterion'] = 'Nearest'
nearest_villages = nearest_villages[gdf.columns]

In [84]:
treatment = pd.concat([gdf, nearest_villages])

In [85]:
treatment['CF_CODE'].nunique()

598

In [86]:
gpd.GeoDataFrame(treatment.drop_duplicates('CF_CODE')).to_file('treatment_villages.shp')